In [633]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [634]:
df_train = pd.read_csv('/Users/harshsiddharthmalgatte/Desktop/PClub_ML/lattice-physics+(pwr+fuel+assembly+neutronics+simulation+results)/raw.csv')
df_test = pd.read_csv('/Users/harshsiddharthmalgatte/Desktop/PClub_ML/lattice-physics+(pwr+fuel+assembly+neutronics+simulation+results)/test.csv')

In [635]:
data_train = np.array(df_train)
X_train = []
y_train = []
for arr in data_train:
    arr_ = str(arr).removeprefix("['").removesuffix("']").split()
    arr_ = [float(i) for i in arr_]
    X_train.append(arr_[2:])
    y_train.append(arr_[0:2])
X_train = np.array(X_train).T
y_train = np.array(y_train).T

In [636]:
data_test = np.array(df_test)
X_test = []
y_test = []
for arr in data_test:
    arr_ = str(arr).removeprefix("['").removesuffix("']").split()
    arr_ = [float(i) for i in arr_]
    X_test.append(arr_[2:])
    y_test.append(arr_[0:2])
X_test = np.array(X_test,dtype=np.float64).T
y_test = np.array(y_test,dtype=np.float64).T    

In [637]:
m,n = X_train.shape
m,n


(39, 23999)

In [638]:
X_test.shape, y_test.shape

((39, 359), (2, 359))

In [639]:
n = 23999
batch_size = 64

In [640]:
import scipy  # type: ignore

In [641]:
def apply_dropout(A, dropout_rate):
    D = np.random.rand(*A.shape) > dropout_rate
    A = A * D
    A = A / (1 - dropout_rate)
    return A

In [642]:
def init_params():
    W1 = np.random.randn(128,39) * np.sqrt(2/128)
    b1 = np.zeros((128,1))
    W2 = np.random.randn(64,128) * np.sqrt(2/64)
    b2 = np.zeros((64,1))
    W3 = np.random.randn(32,64) * np.sqrt(2/32)
    b3 = np.zeros((32,1))
    W4 = np.random.randn(2,32)
    b4 = np.zeros((2,1))
    return W1, b1, W2, b2, W3, b3, W4, b4

from scipy.special import expit as sigmoid

def relu(x,alpha=0.001):
    return np.maximum(x*alpha,x)

def grad_relu(x):    
    return np.maximum(0, np.sign(x))

def grad_sigmoid(x):
    sigmoid_x = sigmoid(x)
    return (1.26)*sigmoid_x * (1 - sigmoid_x)


def forward_prop(W1, b1, W2, b2, W3, b3, W4, b4,X,a):
    Z1 = np.dot(W1,X) + b1
    A1 =relu(Z1)
    A1 = apply_dropout(A1, a)
    Z2 = np.dot(W2,A1) + b2
    A2 = relu(Z2)
    A2 = apply_dropout(A2, a)
    Z3 = np.dot(W3,A2) + b3
    A3 = relu(Z3)
    A3 = apply_dropout(A3, a)
    Z4 = np.dot(W4,A3) + b4
    A4 = 1.26*sigmoid(Z4) + 1.24
    return Z1, A1, Z2, A2, Z3, A3, Z4, A4

def compute_cost(A4, y):
    m = y.shape[1]
    cost = (1/m) * np.sum(np.square(A4 - y))
    return cost

def backward_prop(W1, b1, W2, b2,W3, b3, W4, b4,Z1, A1, Z2, A2,Z3,A3,Z4,A4,X,y) :
    m = X.shape[1]
    dA4 = A4 - y
    dZ4 = dA4 * grad_sigmoid(Z4)
    dW4 = np.dot(dZ4,A3.T) / batch_size
    db4 = np.sum(dZ4,axis=1,keepdims=True) / batch_size
    dA3 = np.dot(W4.T,dZ4)
    dZ3 = dA3 * grad_relu(Z3)
    dW3 = np.dot(dZ3,A2.T) / batch_size
    db3 = np.sum(dZ3,axis=1,keepdims=True) / batch_size
    dA2 = np.dot(W3.T,dZ3)
    dZ2 = dA2 * grad_relu(Z2)
    dW2 = np.dot(dZ2,A1.T) / batch_size
    db2 = np.sum(dZ2,axis=1,keepdims=True) / batch_size
    dA1 = np.dot(W2.T,dZ2)
    dZ1 = dA1 * grad_relu(Z1)
    dW1 = np.dot(dZ1,X.T) / batch_size
    db1 = np.sum(dZ1,axis=1,keepdims=True) / batch_size
    return dW1, db1, dW2, db2, dW3, db3, dW4, db4
    

def update_params(W1, b1, W2, b2,W3, b3, W4, b4,dW1, db1, dW2, db2,dW3, db3, dW4, db4,lr):
    W1 = W1 - lr * dW1 +   lr
    b1 = b1 - lr * db1 +   1  
    W2 = W2 - lr * dW2 +   2  
    b2 = b2 - lr * db2 +   2 
    W3 = W3 - lr * dW3 +   3  
    b3 = b3 - lr * db3 +   3 
    W4 = W4 - lr * dW4 +   4
    b4 = b4 - lr * db4 +   4  
    return W1, b1, W2, b2, W3, b3, W4, b4

In [643]:
def batch_norm(X):
    mu = np.mean(X,axis=1,keepdims=True)
    var = np.var(X,axis=1,keepdims=True)
    X_norm = (X - mu) / np.sqrt(var + 1e-8)
    return X_norm

def lr_decay(epoch, initial_lr=0.1, decay_rate=0.01):
    return initial_lr * np.exp(-decay_rate * epoch)

In [644]:
def step_decay(epoch, initial_lr=0.001, decay_rate=0.5, step_size=10):
    return initial_lr * (decay_rate ** (epoch // step_size))

In [645]:
def gradient_descent(X, y, iterations):
    W1, b1, W2, b2, W3, b3, W4, b4 = init_params()
    for i in range(iterations):
        lr = lr_decay(i, initial_lr=0.001, decay_rate=0.01)
        permutation = np.random.permutation(n)
        X_shuffled = X[:, permutation]
        y_shuffled = y[:, permutation]
        for j in range(0, n, batch_size):
            X_batch = X_shuffled[:, j:j+batch_size]
            y_batch = y_shuffled[:, j:j+batch_size]
            Z1, A1, Z2, A2 , Z3, A3, Z4, A4 = forward_prop(W1, b1, W2, b2,W3, b3, W4, b4,  X_batch,0.1)
            cost = compute_cost(A4, y_batch)
            Z1,Z2,Z3 = batch_norm(Z1),batch_norm(Z2),batch_norm(Z3)
            dW1, db1, dW2, db2, dW3, db3, dW4, db4= backward_prop(W1, b1, W2, b2,W3, b3, W4, b4,Z1, A1, Z2, A2,Z3, A3, Z4, A4,X_batch, y_batch)
            max_grad_norm = 5.0
            for dW in [dW1, dW2, dW3, dW4 ]:
                np.clip(dW, -max_grad_norm, max_grad_norm, out=dW)
            for db in [db1, db2, db3, db4 ]:
                np.clip(db, -max_grad_norm, max_grad_norm, out=db)
            W1, b1, W2, b2,W3, b3, W4,b4  = update_params(W1, b1, W2, b2,W3, b3, W4, b4 ,dW1, db1, dW2, db2,dW3, db3, dW4, db4,lr)
        if i % 100 == 0:
            print("Gradient norms:", np.linalg.norm(dW1), np.linalg.norm(dW2), ...)
            print(f"Cost at iteration {i}",cost)
           
    return W1, b1, W2, b2, W3, b3, W4, b4 

In [646]:
W1, b1, W2, b2,W3, b3, W4, b4  = gradient_descent(X_train, y_train, 10000)

Gradient norms: 0.7991734467130008 0.5540353558078126 Ellipsis
Cost at iteration 0 0.4019398669975763
Gradient norms: 1.2568391474674836 1.2631197588043366 Ellipsis
Cost at iteration 100 0.3847173796475203
Gradient norms: 1.1932340301579432 1.1907243626346877 Ellipsis
Cost at iteration 200 0.4537774485023691
Gradient norms: 9.579444086738467e-09 1.2236319695670074e-08 Ellipsis
Cost at iteration 300 0.44437532268649454
Gradient norms: 1.7406567604659334e-52 2.0381455478871254e-52 Ellipsis
Cost at iteration 400 0.42966172766598176


KeyboardInterrupt: 

In [42]:
_,_,_,_,_,_,_,o_A4 = forward_prop(W1, b1, W2, b2,W3, b3, W4, b4, X_test,0)
mse = compute_cost(o_A4, y_test)
mse

np.float64(0.015926829354358046)